# 5.2 Group-equivariant CNNs

**Review**
- Convolution operator : Linear combination of linear and local translation equivariant operators.
   $$F(x) = \sum_{v=1}^{H^f} \sum_{w=1}^{W^f} \alpha_{vw} C(\theta_{vw})x$$
- Homogeneous space : A set $\Omega$ equipped with a transitive group action, meaning that for any $u,v \in \Omega$ there exists $g \in G$ such that $g\cdot u = v$.

Convolution operator를 Euclidean space 위에서 정의된 시그널을 군 $G$가 작용하는 임의의 homogeneous space 위의 시그널로 보내는 operator로 일반화하자.

Euclidean convolution이 *평행이동*된 필터를 시그널과 매칭시켰던 것과 같이, group convolution은 rotation이나 translating과 같은 group action을 사용하여 필터를 도메인 위에서 움직이도록 할 것이다.
이 경우 group action의 transitivity에 의해 필터는 도메인 위의 임의의 점으로도 움직일 수 있다. 

## Discrete group convolution
먼저 Domain $\Omega$와 group $G$가 모두 discrete한 경우를 고려하자.

예를 들어 3D cubical grid $\Omega = \mathbb{Z}^3$ 위에서 정의된 이미지 $x: \mathbb{Z}^3 \rightarrow \mathbb{R}$이 있다고 하자.
편의를 위해 이미지는 finite cuboid $[W]\times [H]\times [D]\subset \mathbb{Z}^3$ 위에서 정의되고, 편의를 위해 함수 $x$는 이외의 영역에서 0 값을 부여 (zero padding)했다고 하자.

이 데이터의 symmetry를 생각하기 위해 군 $G = \mathbb{Z}^3 \rtimes O_h$를 생각하자. (군의 component들은 각각 distance preserving, orientation preserving transformation을 의미한다.)

이 경우 필터를 translation하는 것 외에, 24가지 종류의 rotation을 하는 것을 생각할 수 있다. 


두 번째 예제로 DNA sequence data를 생각하자.
DNA sequence는 C,G,A,T 네 개의 문자로 구성된다.
이 데이터는 1D grid $\Omega = \mathbb{Z}$ 위의 signal $x:\mathbb{Z} \rightarrow \mathbb{R}^4$의 Ont-Hot encoding 형태로 정의된다. 
이 데이터는 discrete 1D translation symmetry뿐만 아니라, reverse-complement symmetry가 존재한다.
이 경우 대칭군은 $G = \mathbb{Z}\rtimes \mathbb{Z}_2$ 형태로 주어진다.

위 예제들과 같은 상황에서 Group convolution은 $$(x \star \theta)(g) = \sum_{u \in \Omega}x_u\rho(g)\theta_u$$로 정의될 수 있다. 
이는 input signal $x$와 $g \in G$에 의해 $\rho(g)\theta_u = \theta_{g^{-1}u}$로 변환된 필터 $\theta$의 내적으로, 이 operator는 group $G$ 위의 함수가 된다. 

# Group Equivariant Convolutional Networks



## 1. Introduction
Symmetry를 이용하여 sample complexity를 줄인 CNN.
G-convolution이라는 새로운 타입의 네트워크를 사용함.

CNN이 ANN보다 효과적인 이유 중 하나는 *convolutional weight sharing* (출력 뉴런들이 동일한 가중치를 공유하는 현상. ANN은 각각의 출력 뉴런들이 가지는 가중치가 서로 다르다.) 덕분이다.
이 덕분에 파라미터 수를 ANN에 비해 굉장히 효율적으로 사용할 수 있고, 이는 Overfitting의 위험을 줄여준다.
Convolutional weight sharing이 효과적인 이유 중 다른 하나는 데이터가 *translation symmetry*를 가지기 때문이다. 
즉, label function과 data의 분포가 근사적으로 평행이동에 대해 invariant하다.

![convolutional weight sharing](https://github.com/SHlee-TDA/Geometric-Deep-Learning/blob/master/convolutional_weight_sharing.jpeg?raw=true)

![translation symmetry](https://github.com/SHlee-TDA/Geometric-Deep-Learning/blob/master/translation_symmetry2.jpeg?raw=true)

CNN이 ANN보다 효과적인 또 다른 이유는 *deep* 네트워크 구조에 효과적이기 때문이다. 
왜냐하면 CNN의 모든 layer들은 *translation equivariant*를 가지고 있기 때문이다.
즉, 이미지를 평행이동 시키고 그것을 컨볼루션 레이어들에 입력하는 것은 원본 이미지를 동일한 컨볼루션 레이어에 입력한 뒤 그 결과로 나온 feature map들을 동일하게 평행이동시킨것과 같다.
Translation symmetry는 각 컨볼루션 레이어에서 보존되며, 이는 첫 네트워크뿐만 아니라, 심층 네트워크에 가서도 여전히 활용될 수 있다.

![translation equivariance](https://github.com/SHlee-TDA/Geometric-Deep-Learning/blob/master/translation_equivariance.jpeg?raw=true)




Group convolution은 기존 CNN이 가지는 translation symmetry에 대한 성질을 더욱 일반화 하여 회전이나 반사와 같은 더욱 다양한 symmetry들에 대해 비슷한 성질을 만족시키도록 한다.


![group equivariance](https://github.com/SHlee-TDA/Geometric-Deep-Learning/blob/master/group_equivariance.jpeg?raw=true)

## 2. Structured & Equivariant Representations
기존의 DNN은 층이 깊어질 수록 더욱 추상화된 특징 표현을 생성한다.
일반적인 NN theory에서 이 표현 공간은 최소한의 구조인 linear space $\mathbb{R}^n$을 상정한다.

본 논문에서는 표현 공간을 조금 더 일반화하여 linear $G$-space로 생각한다.
여기서 $G$는 선택된 group이다.
즉, 표현 공간의 각 벡터는 그것에 대응하는 *pose*를 가지며, 이는 변환들의 group $G$에 의해 변환될 수 있다.
이러한 추가적인 구조는 데이터를 더욱 효과적으로 모델링할 수 있게 해준다.

표현공간은 그것의 구조를 그것과 연결된 다른 표현공간으로부터 얻을 수 있다.
이렇게 하기 위해서 네트워크나 레이어 $\Phi$는 하나의 표현을 다른 것으로 보내면서 *structure preserving*을 해야한다. 
$G$-space에 대해서, 이것은 $\Phi$는 $$\Phi(T_g x) = T'_g \Phi(x)$$를 만족해야한다.
이를 Equivariance라고 한다.
여기서 $T$와 $T'$가 같을 필요는 없고, 임의의 두 transformation $g,h\in G$에 대하여, $T(gh) = T(g)T(h)$를 만족하면 된다.

만약 $T'_g$가 모든 $g$에 대하여 같은 경우가 invariance의 개념이다.
그러나 일반적인 딥러닝에서 invariance보다 equivariance 개념이 더욱 유용한데, 이는 feature들이 올바른 공간적 구성요소라고 할 때에, 그것이 invariant인지 판별하는 것이 불가능하기 때문이다. 
또한 equivariance는 일반화를 효과적으로 할 수 있게 해준다.
예를들어, 네트워크 $\Phi$는 non-injective할 수 있고, 이는 서로 다른 입력 $x$와 $y$에 대해 동일한 feature map이 출력될 수 있음을 의미한다.
만약 $\Phi$가 equivariant하다면, $G$-transformed input $T_g x$와 $T_g y$에 대해서도 동일한 출력을 얻는다.


## 4. Mathematical Framework
### 4.1 Symmetry Groups
대상의 *symmetry*는 그 대상이 invariant하도록 하는 변환을 의미한다.
예를 들어서, 이미지 데이터에서 grid $\mathbb{Z}^2$를 얻는다면, 그것을 뒤집은 $-\mathbb{Z}^2$는 정확히 같은 집합이므로, 뒤집기는 grid의 symmetry다.

symmetry transformation들의 모임은 자연스럽게 group 구조를 이루고, 이러한 변환들의 군을 대칭군(*symmetry group*)이라 부른다.
예를 들어서, $\mathbb{Z}^2$는 integer translation symmetry에 대한 group이 된다. 
이 군의 operation은 덧셈이다. 
즉, 두 translation의 덧셈은 여전히 translation이다.

### 4.2 The group $p4$
군 $p4$는 정사각형 grid 위의 모든 translation과 임의의 중심에 대한 90도 rotation들의 합성으로 구성되어있다.
이 군의 원소는 세 정수 $r,u,v$에 대해 다음과 같이 표현된다:
$$ g(r,u,v) = \begin{bmatrix}\cos(r\pi/2) & -\sin(r\pi/2) & u \\ \sin(r\pi/2) & \cos(r\pi/2) & v \\ 0 & 0 & 1\end{bmatrix}$$
($0\leq r <4 , (u,v) \in \mathbb{Z}^2$)
이 군의 operation은 행렬곱이다.
이 군의 연산 결과를 표현할 때는 정수 triple $r,u,v$들에 대해 행렬로 표현해 그것을 곱한 다음 그 결과로부터 다시 정수 triple로 표현하는 식으로 한다. ($r$을 얻기 위해 아크탄젠트를 사용한다.)

군 $p4$는 픽셀 좌표 $\mathbb{Z}^2$에 대해 행렬곱으로 작용한다. 
즉, 이미지 데이터 $x$에 대해, 그것의 $(u',v')$ 픽셀은 $g\in p4$에 의해,
$$gx = \begin{bmatrix}\cos(r\pi/2) & -\sin(r\pi/2) & u \\ \sin(r\pi/2) & \cos(r\pi/2) & v \\ 0 & 0 & 1\end{bmatrix}\begin{bmatrix} u' \\ v' \\ 1 \end{bmatrix}$$ 
로 얻어진다.

### 4.3 The group $p4m$
군 $p4m$은 정사각형 grid 위의 모든 translation, mirror reflextion, 90도 회전들의 모임으로 주어진다.
$p4$와 유사하게, 이 군도 4개의 정수쌍 $m,r,u,v$에 의해 다음 행렬로 표현된다.
$$ g(m,r,u,v) = \begin{bmatrix}(-1)^m\cos(r\pi/2) & -(-1)^m\sin(r\pi/2) & u \\ \sin(r\pi/2) & \cos(r\pi/2) & v \\ 0 & 0 & 1\end{bmatrix}$$
($m\in \left\{0,1\right\}, 0\leq r<4, (u,v) \in \mathbb{Z}^2$)

군의 연산은 $p4$의 경우와 유사하게 표현되며, 이 경우 $m$을 얻을 때, $m = \frac{1}{2}(1-\det(g))$를 계산하면 된다.

### 4.4 Functions on groups
기존의 CNN은 함수 $f:\mathbb{Z}^2 \rightarrow \mathbb{R}^K$에 의해 입력받은 이미지를 $K$-channel feature map으로 변환하는 것으로 이해할 수 있다.
여기서 feature map들은 항상 유한한 array에 저장되어야 하지만, 수학적 해석의 편의를 위해 무한한 array로 확장시키자. (이는 유한한 일부 영역 외에 나머지는 모두 zero padding된 것으로 이해하면 된다.)

feature map을 변환하는 것을 transformation $g\in G$에 대한 action으로 이해하자.
이를 다음과 같이 표기할 수 있다:
$$ [L_g f](x) = [f\circ g^{-1}](x) = f(g^{-1}x)$$
이를 해석하면, $g$에 의해 변환된 $L_gf$의 점 $x\in \mathbb{Z}^2$에 대한 feature map은 원래 feature map $f$에서 변환된 이미지 $g^{-1}x$에 대한 feature map과 같다는 것이다.
예를들어, $g$가 translation $t=(u,v)$를 의미한다고 하면, $g^{-1}x$는 translation된 점 $x-t$를 의미한다. 

G-CNN은 위와 같은 관계를 단순히 translation group $\mathbb{Z}^2$ 위에서만 얻는 것이 아닌, 더욱 일반적인 group에서 얻고자 하는 것이다. 
이렇게 하기 위해서는 간단하게 $\mathbb{Z}^2$의 원소였던 $x$를 일반적인 군 $G$의 원소 $h$로 생각하면 된다. 
즉, 우리는 함수 $f$의 domain을 group $G$로 바꾸었다.

예를 들어보자.
$G=p4$ 위의 feature map을 시각화하고자 하면, 원 위의 4개의 패치들과 네 개의 회전을 그린다.
이 그림의 각 픽셀은 rotation coordinate $r$(어떤 패치 위에 픽셀이 그려질지 선택하는 파라미터)과 두 개의 translation coordinate $u,v$(각 패치 위의 픽셀 위치를 나타내는 파라미터)로 구성된다.

만약 90도 회전 $r$을 $p4$ 위의 함수 $f$에 적용하고자 하면, 각 patch를 $r$만큼 회전시켜서 이해하면 된다.

![Figure 1](https://github.com/SHlee-TDA/Geometric-Deep-Learning/blob/master/figure1.png?raw=true)

G-CNN layer를 구체적으로 표현하면 아래의 수식과 같다.
$$[f\star \psi](g) = \sum_{h\in X}\sum_k f_k(h)\psi_k(g^{-1}h).$$
예를들어 $G=p4$라면, $g\in G$는 translation $t$와 rotation $s$의 곱 $g=ts$로 나타낼 수 있고 $L_gL_h = L_{gh}$ 임을 이용하면, $$\psi_k(g^{-1}h) = L_g\psi_k(h) =L_t[L_s\psi_k(h)]$$
가 되어,
$$[f\star \psi](g) = \sum_{h\in X}\sum_k f_k(h)L_t[L_s\psi_k(h)]$$
임을 얻는다.
이는 $p4$-CNN layer를 구성하기 위해, 먼저 filter $\psi_k(h)$를 모든 회전 $s$에 대하여 변환시킨 다음 (즉, $L_s\psi_k$를 만들어냄), 이를 일반적인 CNN처럼 translation 시켜가면서 feature map을 내적으로 계산해나간다 (즉, $f_k(h)L_t[something]$을 계산함.)

## Transform + Convolution approach
Group convolution은 두 가지 스텝으로 구현될 수 있다.
1. Filter transformation step : 기본 필터로부터 rotation 등의 필터를 만들어낸다.
2. Translational convolution step : 보통의 CNN처럼 필터를 translation 시켜가면서 계산한다.

